# 🌾 *MittiSense*
## *An AI-powered multi-agent agriculture assistant delivering crop, soil, weather, and image-based disease advisories for farmers.*
## *Built with ❤️ by: Meet Chaudhari..🧡*


# 🔧 Installing Required Libraries
This section installs the Google Generative AI library (specific version required for the Kaggle Agents course).

In [ ]:
!pip install google-generativeai==0.7.2

# 🔐 Loading the Gemini API Key (Kaggle Secrets)
We securely load the API key using Kaggle's Secrets system.
This avoids exposing the key directly in the notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

# Load the API key from Kaggle Secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Configure Gemini
genai.configure(api_key=api_key)

print("Gemini configured successfully.")

# 🤖 Testing Gemini Connection
We verify that Gemini is configured correctly by generating a simple test message.
If the output prints correctly, the environment is ready.

In [ ]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

# Load API Key
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Configure
genai.configure(api_key=api_key)

# Test Gemini
model = genai.GenerativeModel("models/gemini-2.5-flash")

response = model.generate_content("Reply ONLY with: MittiSense is online.")

print(response.text)

# 🌦️ Weather Information Module
This function retrieves a simple weather-based summary for a given location.
Gemini uses live grounding to incorporate current or typical weather conditions.

In [ ]:
def get_weather_info(location, language="english"):
    # Normalize the language input
    lang = language.strip().lower()

    # Language rules for the model
    if lang == "hindi":
        lang_rule = "Write the entire weather summary in simple Hindi only."
    elif lang == "hinglish" or lang == "mixed":
        lang_rule = "Write the weather summary in Hinglish (Hindi sentences with English keywords)."
    else:
        lang_rule = "Write the weather summary in simple English only."

    prompt = f"""
You are a weather advisory assistant.

LANGUAGE MODE:
{lang_rule}

TASK:
Provide a short and practical weather summary.

LOCATION:
{location}

INCLUDE:
- Current weather condition
- Temperature
- Rain possibility
- Any warnings important for agriculture

Keep it short, simple, and easy to understand.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Testing Weather Information Function
We test the weather function with a sample location to confirm that weather summaries work.


In [ ]:
print(get_weather_info("Gujarat", language="hindi"))

# 🌩️ Weather-Aware MittiSense Advisory
This enhanced version of MittiSense combines crop details, soil behavior, and live weather reasoning
to produce a more accurate, context-aware advisory.

In [ ]:
def mittisense_full_report(crop, location, soil, season, notes="", language="english"):
    # Normalize & interpret language
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the entire advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the advisory in Hinglish (Hindi sentences with English keywords)."
    else:
        lang_rule = "Write the advisory in simple English only."

    # Get weather info in the SAME language
    weather = get_weather_info(location, language=language)

    # Build prompt
    prompt = f"""
You are MittiSense, an agricultural advisory assistant.

LANGUAGE MODE:
{lang_rule}

DO NOT assume the user is a farmer.
DO NOT use greetings like "kisan bhai" or "farmer".
Use a neutral tone.

INPUT DETAILS:
- Crop: {crop}
- Location: {location}
- Soil: {soil}
- Season: {season}
- User Notes: {notes}

TODAY'S LOCAL WEATHER:
{weather}

TASK:
Using the weather + crop + soil + season + notes:
Provide a detailed advisory including:

1. Weather impact on the crop today
2. What actions to avoid today
3. Soil and nutrient recommendations
4. Irrigation guidance based on today's weather
5. Disease and pest expectations
6. Step-by-step actions for the next 7 days
7. A final short summary

RULES:
- Match the selected language EXACTLY.
- Use clear, simple sentences.
- No chemical brand names.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Testing the Weather-Aware Advisory
We generate a full advisory including weather impact and management suggestions.

In [ ]:
print(mittisense_full_report(
    "Cotton", "Surat", "Black Soil", "Kharif",
    "Plant growth is slow",
    language="hinglish"
))

# 🌱 Soil Intelligence Module for MittiSense
This section adds soil understanding to MittiSense.  
Different soils behave differently, so this module helps customize irrigation, fertilizer,
and risk assessments based on soil type.

In [ ]:
def get_soil_profile(soil_type: str) -> str:
    soil_type_lower = soil_type.strip().lower()

    if "black" in soil_type_lower:
        return (
            "Black soil has high moisture-holding capacity and is rich in clay. "
            "It suits cotton, wheat, and pulses but can get waterlogged easily. "
            "Irrigation should be moderate and well-timed."
        )
    elif "red" in soil_type_lower:
        return (
            "Red soil is low in organic matter and nutrients. It drains quickly, "
            "so it requires frequent irrigation and higher organic manure input."
        )
    elif "alluvial" in soil_type_lower:
        return (
            "Alluvial soil is fertile and ideal for crops like wheat, rice, sugarcane, "
            "and vegetables. It responds very well to balanced NPK fertilization."
        )
    elif "sandy" in soil_type_lower:
        return (
            "Sandy soil drains very fast and has low nutrient retention. "
            "It needs frequent light irrigation and organic matter to improve structure."
        )
    elif "loam" in soil_type_lower or "loamy" in soil_type_lower:
        return (
            "Loamy soil is a balanced mixture of sand, silt, and clay. "
            "It is excellent for most crops due to good drainage and nutrient holding."
        )
    elif "clay" in soil_type_lower:
        return (
            "Clay soil holds a lot of water and becomes compact. "
            "Irrigation should be less frequent, and good drainage is essential."
        )
    else:
        return (
            "Soil type not clearly identified. Provide general recommendations: "
            "balanced irrigation, organic matter addition, and standard NPK usage."
        )

# 🌾 Soil-Aware MittiSense Advisory
This function generates a full advisory based on:
- Crop
- Location
- Season
- Soil type (using the soil profile module)

This makes MittiSense far more accurate and personalized.


In [ ]:
import google.generativeai as genai

def mittisense_soil_aware_report(crop, location, soil, season, notes="", language="english"):
    # Normalize language input
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the entire advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the advisory in Hinglish (Hindi mixed with English keywords)."
    else:
        lang_rule = "Write the advisory in simple English only."

    soil_profile = get_soil_profile(soil)

    prompt = f"""
You are MittiSense, an agricultural advisory assistant.

LANGUAGE MODE:
{lang_rule}

Do NOT assume the user is a farmer.
Do NOT use greetings like "kisan bhai" or "farmer".
Use a neutral, professional tone.

USER INPUT:
- Crop: {crop}
- Location: {location}
- Soil Type: {soil}
- Season: {season}
- Notes: {notes}

IMPORTANT SOIL INFORMATION:
{soil_profile}

TASK:
Based on the soil, crop, and season, provide:

1. Soil-specific risks (e.g., waterlogging, nutrient loss, drought sensitivity)
2. Soil preparation steps suitable for this soil type
3. Sowing guidance (only if relevant)
4. Irrigation schedule adapted to this soil behavior
5. Fertilizer and organic matter guidance (natural, no chemical brand names)
6. Issues the user should monitor
7. A short, clear final summary

RULES:
- Match the selected language EXACTLY.
- Structured headings + bullet points.
- No chemical brand names.
- No greetings.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ✅ Testing the Soil-Aware MittiSense Report
Run the test below to verify that the soil intelligence is working correctly.
The response should contain soil-specific advice (based on Black Soil in this example).


In [ ]:
print(mittisense_soil_aware_report(
    "Cotton", "Akola", "Black Soil", "Kharif",
    "Plants growing slowly",
    language="english"
))

# 🌿 Basic MittiSense Advisory (Initial Version)
This version generates a general agricultural advisory based on crop, location, season,
and farmer notes. It does not yet include soil intelligence or weather integration.

In [ ]:
def agent_core_advisory(crop, location, soil, season, notes="", language="english"):
    # Normalize language input
    lang = language.strip().lower()

    # Language selection + greeting rule
    if lang == "hindi":
        lang_rule = (
            "Write the entire advisory in simple Hindi only. "
            "Do NOT use English except for crop names. "
            "Use a NEUTRAL greeting like 'Namaste' ONLY if needed. "
            "NEVER address the user as a farmer."
        )
    elif lang == "hinglish" or lang == "mixed":
        lang_rule = (
            "Write the advisory in Hinglish (Hindi sentence flow + English keywords). "
            "Use a neutral greeting like 'Hello' if needed. "
            "NEVER address the user as a farmer."
        )
    else:  # default English
        lang_rule = (
            "Write the entire advisory in simple English only. "
            "Use a neutral greeting like 'Hello' only if needed. "
            "NEVER address the user as a farmer."
        )

    soil_profile = get_soil_profile(soil)
    weather_info = get_weather_info(location)

    prompt = f"""
You are Agent-1: MittiSense Core Advisory.

LANGUAGE MODE:
{lang_rule}

USER INPUT:
- Crop: {crop}
- Location: {location}
- Soil: {soil}
- Season: {season}
- Notes: {notes}

SOIL PROFILE:
{soil_profile}

WEATHER TODAY:
{weather_info}

TASK:
Provide a complete advisory with the following sections:

1. Soil considerations  
2. Weather impact  
3. Irrigation guidance  
4. Nutrient guidance (natural and safe)  
5. Expected pests & diseases  
6. Safe natural actions  
7. What to monitor this week  
8. Short summary  

IMPORTANT RULES:
- NEVER address the user as 'farmer', 'kisan', 'grower', etc.
- Greeting must be neutral if used (e.g., 'Hello').
- No chemical brand names.
- Keep the language simple, safe, natural and aligned with LANGUAGE MODE.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Testing the Basic Advisory Function
We run a sample test to ensure the initial MittiSense advisory works as expected.

In [ ]:
# ▶️ Testing Agent-1 (Language-Enabled Core Advisory)

result = agent_core_advisory(
    crop="Wheat",
    location="Gujarat",
    soil="Black Soil",
    season="Rabi",
    notes="Leaves turning yellow early",
    language="english"   # or "hindi" or "hinglish"
)

print(result)

# 🦠 Crop Disease Predictor Module
This module predicts the major diseases and pest risks for a given crop and season.
The model uses agricultural domain patterns to suggest:
- Common diseases
- Expected symptoms
- Early preventive steps
- Risk factors based on season

In [ ]:
def predict_crop_diseases(crop: str, season: str, location: str = "", notes: str = "", language="english"):
    # Normalize language
    lang = language.strip().lower()

    # Language rules
    if lang == "hindi":
        lang_rule = "Write the entire disease advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the advisory in Hinglish (Hindi sentences with English keywords)."
    else:
        lang_rule = "Write the disease advisory in simple English only."

    prompt = f"""
You are MittiSense, an agricultural disease and pest prediction assistant.

LANGUAGE MODE:
{lang_rule}

TASK:
Based on the crop, season, and user-provided notes, predict:

1. Most probable diseases for this crop
2. Common pests during this season
3. Early symptoms the user should look for
4. Weather or soil conditions that increase disease/pest risk
5. Natural & safe preventive steps (NO chemical brand names)
6. Simple do/don't guidelines

INPUT DETAILS:
- Crop: {crop}
- Season: {season}
- Location: {location}
- Notes/Symptoms: {notes}

RULES:
- Match the selected language EXACTLY.
- Keep tone neutral, simple, and professional.
- No chemical brand names.
- No farmer-targeted greetings.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Testing the Crop Disease Predictor
We test the disease module with a common crop-season combination.

In [ ]:
print(predict_crop_diseases(
    crop="Cotton",
    season="Kharif",
    location="Maharashtra",
    notes="Patton par safed daag",
    language="hindi"
))

# 📦 Structured JSON Output for MittiSense
This module forces Gemini to return responses in clean JSON format.
Each advisory section is placed into its own key:
- soil_analysis
- weather_impact
- irrigation
- fertilizer_plan
- disease_risks
- actions
- summary

This is essential for building apps and agent pipelines.


In [ ]:
def mittisense_json_report(crop, location, soil, season, notes="", language="english"):
    # Normalize language
    lang = language.strip().lower()

    # Language rules for JSON values
    if lang == "hindi":
        lang_rule = "Write all JSON **values** in simple Hindi only. JSON keys must remain in English."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write all JSON **values** in Hinglish (Hindi sentences with English keywords). JSON keys must remain in English."
    else:
        lang_rule = "Write all JSON values in simple English only. JSON keys must remain in English."

    # Use language-aware weather
    weather_summary = get_weather_info(location, language=language)
    soil_profile = get_soil_profile(soil)

    prompt = f"""
You are MittiSense, an agricultural advisory agent.

LANGUAGE MODE:
{lang_rule}

Your task is to output CLEAN JSON ONLY.
No explanation. No markdown. No text outside JSON.

INPUT DATA:
- Crop: {crop}
- Location: {location}
- Soil: {soil}
- Season: {season}
- Notes: {notes}

SOIL PROFILE:
{soil_profile}

LOCAL WEATHER:
{weather_summary}

Return a JSON object with the EXACT keys below:

{{
  "soil_analysis": "",
  "weather_impact": "",
  "irrigation_recommendation": "",
  "fertilizer_plan": "",
  "disease_and_pest_risks": "",
  "actions_for_next_7_days": "",
  "summary": ""
}}

RULES:
- JSON output ONLY.
- No text before or after the JSON.
- JSON must be valid.
- Values must follow the selected language.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Testing the JSON-based MittiSense Report
We check if the output is clean, valid JSON.


In [ ]:
print(mittisense_json_report(
    "Cotton", "Gujarat", "Black Soil", "Kharif",
    notes="Leaves curling",
    language="english"
))

# 🤖 Multi-Agent System for MittiSense
This section builds a 3-agent pipeline:

1. **Agent 1 — Core Advisory Agent**  
   Generates the initial advisory using soil, weather, crop, and user notes.

2. **Agent 2 — Disease Expert Agent**  
   Reviews the advisory and adds/updates disease + pest risks.

3. **Agent 3 — Final Reviewer Agent**  
   Checks for unsafe recommendations, improves clarity, ensures safety and correctness.

The final output is a polished, validated, multi-agent agricultural advisory.

In [ ]:
def agent_core_advisory(crop, location, soil, season, notes="", language="english"):
    # LANGUAGE MODE
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the entire advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the advisory in Hinglish (Hindi with English keywords)."
    else:
        lang_rule = "Write the advisory in simple English only."

    soil_profile = get_soil_profile(soil)
    weather_info = get_weather_info(location, language=language)

    prompt = f"""
You are Agent-1: MittiSense Core Advisory Agent.

LANGUAGE MODE:
{lang_rule}

Do NOT assume the user is a farmer.
Do NOT use greetings.
Maintain a neutral, professional tone.

INPUT:
- Crop: {crop}
- Location: {location}
- Soil: {soil}
- Season: {season}
- User Notes: {notes}

SOIL PROFILE:
{soil_profile}

WEATHER SUMMARY:
{weather_info}

TASK:
Provide a clear agricultural advisory including:
1. Soil-related considerations
2. Weather-impact & precautions
3. Irrigation strategy (safe, general)
4. Nutrient management (no chemical brands)
5. Common pests & diseases for THIS crop + season
6. Safe natural preventive steps
7. What the user should monitor this week
8. A short summary (4–5 lines)

RULES:
- Follow selected language strictly.
- Keep it simple and structured.
- No chemical product names.
- Stay agricultural (no indoor-plant style advice).
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    return model.generate_content(prompt).text

In [ ]:
def agent_disease_expert(advisory_text, crop, season, language="english"):
    # Language setup
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the disease & pest section in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the disease & pest section in Hinglish."
    else:
        lang_rule = "Write the disease & pest section in simple English only."

    prompt = f"""
You are Agent-2: Disease & Pest Specialist.

LANGUAGE MODE:
{lang_rule}

TASK:
- Read the advisory below.
- Improve, correct, and expand the pest + disease section.
- Add missing diseases for this crop & season.
- Add early symptoms to monitor.
- Provide safe, natural preventive steps.
- Output ONLY the improved disease/pest section.

CROP: {crop}
SEASON: {season}

ADVISORY PROVIDED:
{advisory_text}

RULES:
- No chemical brand names.
- Keep the tone professional.
- Match the selected language exactly.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    return model.generate_content(prompt).text

In [ ]:
def agent_final_reviewer(core_text, pest_text, language="english"):
    # Language mode
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the final advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the final advisory in Hinglish (Hindi + English mix)."
    else:
        lang_rule = "Write the final advisory in simple English only."

    prompt = f"""
You are Agent-3: MittiSense Final Reviewer & Safety Checker.

LANGUAGE MODE:
{lang_rule}

Your job:
1. Combine Agent-1 (core) + Agent-2 (disease).
2. Ensure correctness, clarity, and safety.
3. Remove irrelevant or indoor-plant style advice.
4. Keep all recommendations natural, cultural, or mechanical.
5. Keep focus strictly on agriculture and the given crop.

CONTENT TO COMBINE:

--- CORE ADVISORY ---
{core_text}

--- DISEASE & PEST EXPERT OUTPUT ---
{pest_text}

Produce the FINAL ADVISORY with sections:
1. Crop Situation Overview
2. Soil & Weather Considerations
3. Irrigation Plan
4. Nutrient Guidance (no chemicals)
5. Pest & Disease Risks
6. Safe Natural Preventive Actions
7. Weekly Monitoring Plan
8. Final Summary

RULES:
- Match selected language.
- No chemical brands.
- No greetings.
- Neutral, professional agricultural tone.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    return model.generate_content(prompt).text

# ▶️ Testing the Full Multi-Agent MittiSense Pipeline
We run all 3 agents step-by-step and generate the final validated advisory.


In [ ]:
# Step 1: Core Advisory
core = agent_core_advisory(
    crop="Cotton",
    location="Surat, Gujarat",
    soil="Black Soil",
    season="Kharif",
    notes="Leaves have small white spots",
    language="english"
)

# Step 2: Disease Expert Review
disease = agent_disease_expert(
    advisory_text=core,
    crop="Cotton",
    season="Kharif",
    language="english"
)

# Step 3: Final Reviewer
final_report = agent_final_reviewer(
    core_text=core,
    pest_text=disease,
    language="english"
)

print(final_report)

# 🚜 MittiSense End-to-End Agent Pipeline

This section creates a single orchestrator function that:
1. Calls Agent 1 (core advisory),
2. Calls Agent 2 (disease & pest specialist),
3. Calls Agent 3 (final reviewer & safety checker),

and returns all three outputs in one go.  
This makes it easy to use MittiSense as a single function in other parts of the notebook or future apps.

In [ ]:
def run_mittisense_pipeline(crop, location, soil, season, notes="", language="english"):
    """
    Runs the full multilingual MittiSense pipeline:
    1. Agent-1 → Core Advisory
    2. Agent-2 → Disease/Pest Expert
    3. Agent-3 → Final Reviewer
    Returns all outputs in a dictionary (clean + structured).
    """

    # --- Agent 1 ---
    core_advisory = agent_core_advisory(
        crop=crop,
        location=location,
        soil=soil,
        season=season,
        notes=notes,
        language=language
    )

    # --- Agent 2 ---
    disease_expert_output = agent_disease_expert(
        advisory_text=core_advisory,
        crop=crop,
        season=season,
        language=language
    )

    # --- Agent 3 ---
    final_advisory = agent_final_reviewer(
        core_text=core_advisory,
        pest_text=disease_expert_output,
        language=language
    )

    return {
        "core_advisory": core_advisory,
        "disease_expert": disease_expert_output,
        "final_advisory": final_advisory
    }

# ▶️ Testing the Full MittiSense Pipeline (Single Call)

Now we test the orchestrator `run_mittisense_pipeline` with a cotton example.
This should run all 3 agents internally and return:
- core advisory
- disease expert review
- final polished advisory

In [ ]:
result = run_mittisense_pipeline(
    crop="Cotton",
    location="Surat, Gujarat",
    soil="Black Soil",
    season="Kharif",
    notes="Leaves have white spots",
    language="english"
)

print(result["final_advisory"])

# 📨 Agent 4 – Short Summary Generator

This agent takes the final MittiSense advisory and converts it into:
- a short, easy-to-read summary
- limited number of lines
- suitable for quick viewing, SMS, or WhatsApp-style messages.

It does NOT add new advice, it only compresses the existing one.


In [ ]:
def mittisense_short_summary(text, max_points=5, language="english"):
    # Normalize language selection
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the summary in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the summary in Hinglish (Hindi mixed with English keywords)."
    else:
        lang_rule = "Write the summary in simple English only."

    prompt = f"""
You are Agent-4: MittiSense Summary Generator.

LANGUAGE MODE:
{lang_rule}

TASK:
Summarize the following agricultural advisory into {max_points} short bullet points.
Keep it simple, clear, and helpful.
Do NOT add new information.
Do NOT use greetings.
Do NOT change the meaning.
Bullet points only.

TEXT TO SUMMARIZE:
{text}
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)

    return response.text

# ▶️Testing the Short Summary Agent (Agent 4)
# 🌱 Step 1: Run the full MittiSense pipeline 
We now generate a short summary from the final MittiSense advisory.
This is useful for quick display or messaging.


In [ ]:
pipeline = run_mittisense_pipeline(
    crop="Cotton",
    location="Surat, Gujarat",
    soil="Black Soil",
    season="Kharif",
    notes="Leaves have small white spots",
    language="english"
)

summary = mittisense_short_summary(
    text=pipeline["final_advisory"],
    max_points=5,
    language="english"
)

print(summary)

## 🔄 Step 2: Final Output Compilation

This step collects and displays the full results generated by the MittiSense agent pipeline.

It includes:

- Agent 1 – Core Advisory  
- Agent 2 – Disease & Pest Expert  
- Agent 3 – Final Reviewer  
- Agent 4 – Short Summary  

The next cell prints the full combined advisory in a structured format.


In [ ]:
pipeline_result = run_mittisense_pipeline(
    crop="Cotton",
    location="Surat, Gujarat",
    soil="Black Soil",
    season="Kharif",
    notes="Leaves have small white spots",
    language="english"    # change to hindi / hinglish anytime
)
core_output = pipeline_result["core_advisory"]
disease_output = pipeline_result["disease_expert"]
final_output = pipeline_result["final_advisory"]
summary_output = mittisense_short_summary(
    text=final_output,
    max_points=5,
    language="english"     # same language as pipeline
)

In [ ]:
print("===== CORE ADVISORY (Agent 1) =====")
print(core_output)

print("\n===== DISEASE & PEST EXPERT (Agent 2) =====")
print(disease_output)

print("\n===== FINAL ADVISORY (Agent 3) =====")
print(final_output)

print("\n===== SHORT SUMMARY (Agent 4) =====")
print(summary_output)

## 🔍 Step 3 — Display & Export Final MittiSense Advisory

Now that all agent outputs are generated, this step will:

- Display each advisory cleanly  
- Combine everything into a single structured file  
- Allow exporting to `.txt` or `.json`  


In [ ]:
# Input details for the advisory system
crop = "Cotton"
location = "Surat, Gujarat"
soil = "Black Soil"
season = "Kharif"
notes = "Leaves have small white spots"

# Language for entire pipeline
LANG = "english"     # change to "hindi" or "hinglish"

In [ ]:
import json

# --- DO NOT reassign crop/location/... if already defined ---
# They are already used in the pipeline

export_data = {
    "crop": crop,
    "location": location,
    "soil": soil,
    "season": season,
    "notes": notes,
    "language": LANG,
    "core_advisory": core_output,
    "disease_expert": disease_output,
    "final_advisory": final_output,
    "short_summary": summary_output
}

# -------- DISPLAY --------
print("===== MITTISENSE FINAL REPORT =====\n")
print(final_output)

print("\n===== SHORT SUMMARY =====")
print(summary_output)       # summary_output is already a string


# -------- SAVE JSON (UTF-8 for Hindi/Hinglish) --------
with open("mittisense_report.json", "w", encoding="utf-8") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

# -------- SAVE TXT --------
with open("mittisense_report.txt", "w", encoding="utf-8") as f:
    f.write("===== FINAL MITTISENSE ADVISORY =====\n\n")
    f.write(final_output)
    f.write("\n\n===== SHORT SUMMARY =====\n")
    f.write(summary_output)   # no loop (it's a single string)

print("\nFiles created: mittisense_report.json , mittisense_report.txt")

# 🧪 Agent-5 — Natural Fertilizer & Nutrient Advisory

This agent provides **safe, natural, and chemical-free nutrient guidance** for farmers.  
It reads soil type, organic matter levels, crop type, and visible symptoms, then identifies:

- Likely nutrient deficiencies  
- Natural fertilizers to correct each deficiency  
- Long-term soil-health improvement practices  
- Simple farmer-friendly suggestions  

The advisory is always:
- 100% natural  
- Easy to understand  
- Safe for soil, crop, and environment  

Use this agent whenever the farmer reports **yellowing, spots, weak growth, or low organic matter**.

In [ ]:
def agent_fertilizer_advisory(crop, soil_type, season, organic_matter, symptoms, language="english"):
    # Normalize language input
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the fertilizer and nutrient advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the fertilizer and nutrient advisory in Hinglish (Hindi mixed with English keywords)."
    else:
        lang_rule = "Write the fertilizer and nutrient advisory in simple English only."

    prompt = f"""
You are Agent-5: MittiSense Natural Nutrient & Fertilizer Advisor.

LANGUAGE MODE:
{lang_rule}

TASK:
Provide a nutrient advisory using ONLY natural, organic, chemical-free methods.

INPUT:
- Crop: {crop}
- Soil Type: {soil_type}
- Season: {season}
- Organic Matter Level: {organic_matter}
- Observed Symptoms: {symptoms}

OUTPUT REQUIREMENTS:
- Identify possible nutrient deficiencies based on symptoms.
- Recommend natural, organic inputs only:
  (compost, FYM, vermicompost, green manure, jeevamrut, mulching, bio-microbes).
- Give simple explanations for each recommendation.
- Provide preventive soil-health practices.
- No chemical fertilizers or chemical brand names.
- No greetings. Keep tone neutral and professional.
- Use the selected language strictly.

Write the final advisory in short, clear paragraphs.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)

    return response.text

In [ ]:
fert_output = agent_fertilizer_advisory(
    crop="Cotton",
    soil_type="Black Soil",
    season="Kharif",
    organic_matter="Low",
    symptoms="Leaves have small white spots and some yellowing",
    language="english"
)

print(fert_output)

# 🌦️ Agent-6: Weather & Climate Advisory

Agent-6 analyzes real-time environmental conditions and gives crop-specific, season-aware climate guidance.  
It focuses on:

- Weather impact on current crop condition  
- Irrigation adjustments based on temperature, humidity & rainfall  
- Heat stress / rain alerts / wind alerts  
- Weather-driven pest & disease risks  
- Safe & practical field-level actions  

This agent **never suggests chemicals** — only safe, farmer-ready advice.

In [ ]:
def agent_weather_advisory(crop, location, season, temp_c, humidity, rain_mm, wind_kmph, language="english"):
    # Normalize language mode
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the weather advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the weather advisory in Hinglish (Hindi mixed with English keywords)."
    else:
        lang_rule = "Write the weather advisory in simple English only."

    prompt = f"""
You are Agent-6: MittiSense Weather & Climate Advisor.

LANGUAGE MODE:
{lang_rule}

TASK:
Give a short, clear, crop-specific weather advisory.

INPUT DETAILS:
- Crop: {crop}
- Location: {location}
- Season: {season}

CURRENT WEATHER:
- Temperature: {temp_c}°C
- Humidity: {humidity}%
- Rainfall: {rain_mm} mm
- Wind Speed: {wind_kmph} km/h

OUTPUT REQUIREMENTS:
1. Explain how today's weather conditions affect the crop.
2. Give irrigation adjustment guidance based on weather.
3. Mention weather-driven pest/disease risks.
4. Add heat, rainfall, humidity, or wind warnings if needed.
5. Only natural and safe actions.
6. No chemical pesticide or fertilizer recommendations.
7. No greetings.
8. Keep language exactly as per LANGUAGE MODE.

Write the advisory in short, clear paragraphs.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)

    return response.text

# 🔍 Test Agent-6 (Weather Advisory)

This cell runs Agent-6 with sample weather data to verify the advisory output.  
You can modify temperature, humidity, rainfall, and wind speed anytime to see updated results.

In [ ]:
weather_output = agent_weather_advisory(
    crop="Cotton",
    location="Surat, Gujarat",
    season="Kharif",
    temp_c=38,
    humidity=45,
    rain_mm=2,
    wind_kmph=12,
    language="english"
)

print(weather_output)

# 🌱 Agent-7: Soil Health Advisory  
This agent analyzes soil type, pH, moisture, drainage, and organic matter  
and provides natural, chemical-free guidance — strictly in English.

In [ ]:
def agent_soil_advisory(soil_type, moisture, ph, drainage, organic_matter, language="english"):
    # Normalize language mode
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the soil advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the soil advisory in Hinglish (Hindi mixed with English keywords)."
    else:
        lang_rule = "Write the soil advisory in simple English only."

    prompt = f"""
You are Agent-7: MittiSense Soil Health Advisor.

LANGUAGE MODE:
{lang_rule}

TASK:
Provide a natural, chemical-free soil health advisory.

INPUT DETAILS:
- Soil Type: {soil_type}
- Moisture Level: {moisture}
- pH Value: {ph}
- Drainage Quality: {drainage}
- Organic Matter Level: {organic_matter}

OUTPUT REQUIREMENTS:
1. Interpret the soil condition clearly.
2. Explain risks (compaction, nutrient lock, waterlogging, dryness).
3. Recommend ONLY natural solutions:
   - compost
   - FYM
   - vermicompost
   - green manure
   - mulching
   - jeevamrut
   - microbial boosters
4. No chemical suggestions.
5. No greetings.
6. Keep tone neutral and farmer-friendly.
7. Strictly follow LANGUAGE MODE.

Write the final soil advisory in short, clear paragraphs.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)

    return response.text

# ▶️ Testing Agent-7 (Soil Advisory)

Running soil advisory with example soil conditions.

In [ ]:
soil_output = agent_soil_advisory(
    soil_type="Black Soil",
    moisture="Medium",
    ph=7.2,
    drainage="Moderate",
    organic_matter="Low",
    language="english"
)

print(soil_output)

# 🌿 Agent-8: Organic Nutrient & Fertility Advisor

This agent provides crop-specific nutrient recommendations based ONLY on organic inputs
like compost, FYM, jeevamrut, green manure, mulching, and microbial solutions.

No chemicals.  
Farmer-friendly language.  
Fully aligned with natural farming.

In [ ]:
def agent_organic_nutrient_booster(soil_type, organic_matter, crop, symptoms="", language="english"):
    # Normalize language mode
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the nutrient booster advisory in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the nutrient booster advisory in Hinglish (Hindi mixed with English keywords)."
    else:
        lang_rule = "Write the nutrient booster advisory in simple English only."

    prompt = f"""
You are Agent-8: MittiSense Organic Nutrient & Soil Fertility Booster Advisor.

LANGUAGE MODE:
{lang_rule}

TASK:
Provide a natural, organic-only fertility improvement advisory.

INPUT DETAILS:
- Soil Type: {soil_type}
- Organic Matter Level: {organic_matter}
- Crop: {crop}
- Symptoms/Notes: {symptoms}

OUTPUT REQUIREMENTS:
1. Briefly explain the soil fertility status.
2. Suggest natural nutrient boosters:
   - compost
   - FYM
   - vermicompost
   - green manure crops
   - bio-slurry
   - jeevamrut
   - mulch layer additions
   - beneficial microbes
3. Provide organic ways to improve soil structure.
4. Suggest weekly soil-health maintenance steps.
5. No chemical fertilizers or chemical brand names.
6. No greetings.
7. Clear, simple sentences.
8. Follow LANGUAGE MODE strictly.

Write the final advisory in small, clear paragraphs.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Testing Agent-8 (Organic Nutrient Advisory)

We now run the nutrient advisory on cotton crop conditions.

In [ ]:
organic_output = agent_organic_nutrient_booster(
    soil_type="Black Soil",
    organic_matter="Low",
    crop="Cotton",
    symptoms="Slow growth and mild yellowing",
    language="english"
)

print(organic_output)

# 🌱 Agent-9: Historical Crop Pattern Advisor
This agent analyses traditional cropping patterns for the selected region and crop,
and provides natural, safe recommendations in English only.

In [ ]:
def agent_historical_pattern_analyzer(crop, location, season, notes="", language="english"):
    # Normalize language input
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the analysis and warnings in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the analysis and warnings in Hinglish (Hindi with English mix)."
    else:
        lang_rule = "Write the analysis and warnings in simple English only."

    prompt = f"""
You are Agent-9: MittiSense Historical Pattern Analyzer & Early Warning System.

LANGUAGE MODE:
{lang_rule}

TASK:
Based on historical patterns for the given crop, location, and season, identify:
1. Past weather patterns that commonly affected this crop.
2. Past pest & disease outbreaks typical during this season.
3. Early signs the user should observe in the coming 7–14 days.
4. Environmental triggers that increase risk.
5. Natural & safe preventive actions.
6. Avoid chemical fertilizers or pesticides.
7. No greetings. Maintain a neutral tone.

INPUT:
- Crop: {crop}
- Location: {location}
- Season: {season}
- Notes/Symptoms: {notes}

OUTPUT FORMAT:
- Start with a short pattern summary.
- Then give 3–5 early warning signals.
- Then provide natural preventive steps.
- Keep it simple, clear, and structured.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Test Agent-9 (Historical Crop Pattern Advisory)

In [ ]:
hist_output = agent_historical_pattern_analyzer(
    "Cotton", "Surat, Gujarat", "Kharif",
    notes="Leaves pe white spots",
    language="hinglish"
)

print(hist_output)

# 🌾 Agent-10: Yield Prediction Advisor
This agent generates a simple, approximate, safe yield expectation based on crop,
soil, season, and current crop symptoms — using natural, non-chemical perspective.
English only.

In [ ]:
def agent_yield_estimator(crop, soil_type, season, weather_summary, symptoms="", language="english"):
    # Normalize language mode
    lang = language.strip().lower()

    if lang == "hindi":
        lang_rule = "Write the yield estimation and outlook in simple Hindi only."
    elif lang in ["hinglish", "mixed"]:
        lang_rule = "Write the yield estimation in Hinglish (Hindi with English mix)."
    else:
        lang_rule = "Write the yield estimation in simple English only."

    prompt = f"""
You are Agent-10: MittiSense Yield Estimator & Production Outlook.

LANGUAGE MODE:
{lang_rule}

TASK:
Give a realistic yield trend estimate and production outlook using only natural farm-safe guidance.

INPUT DETAILS:
- Crop: {crop}
- Soil Type: {soil_type}
- Season: {season}
- Weather Summary: {weather_summary}
- Symptoms/Notes: {symptoms}

OUTPUT REQUIREMENTS:
1. Predict overall yield trend (e.g., low / moderate / good potential).
2. Explain the reasons (weather, soil condition, symptoms).
3. Mention factors that may improve yield.
4. Mention factors that may reduce yield.
5. Give natural, safe, soil-health and plant-health steps to improve yield.
6. No chemicals or brand-name inputs.
7. No greetings. Neutral, professional tone.
8. Strictly follow LANGUAGE MODE.

Write the output in clean, short paragraphs.
"""

    model = genai.GenerativeModel("models/gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text

# ▶️ Test Agent-10 (Yield Prediction Advisory)

In [ ]:
yield_output = agent_yield_estimator(
    "Cotton", "Black Soil", "Kharif",
    weather_summary="Hot afternoon, light humidity, slight rain possibility",
    symptoms="Leaves pe white spots",
    language="hinglish"
)

print(yield_output)

# 🚀 MittiSense Master Pipeline

In [ ]:
def mittisense_master_pipeline(
    crop,
    location,
    soil,
    season,
    notes="",
    temp_c=32,
    humidity=50,
    rain_mm=0,
    wind_kmph=5,
    moisture="Medium",
    ph=7.0,
    drainage="Moderate",
    organic_matter="Medium",
    language="english",
    summary_points=5
):
    """
    FULL MULTI-AGENT MITTISENSE PIPELINE
    Runs Agents 1–10 and returns all outputs in a single dictionary.
    """

    # -------------------------
    # AGENT 1 → Core Advisory
    # -------------------------
    core = agent_core_advisory(
        crop=crop,
        location=location,
        soil=soil,
        season=season,
        notes=notes,
        language=language
    )

    # -------------------------
    # AGENT 2 → Disease Expert
    # -------------------------
    disease = agent_disease_expert(
        advisory_text=core,
        crop=crop,
        season=season,
        language=language
    )

    # -------------------------
    # AGENT 3 → Final Reviewer
    # -------------------------
    final_advisory = agent_final_reviewer(
        core_text=core,
        pest_text=disease,
        language=language
    )

    # -------------------------
    # AGENT 4 → Short Summary
    # -------------------------
    short_summary = mittisense_short_summary(
        text=final_advisory,
        max_points=summary_points,
        language=language
    )

    # -------------------------
    # AGENT 5 → Natural Fertilizer Advisory
    # -------------------------
    fertilizer_advice = agent_fertilizer_advisory(
        crop=crop,
        soil_type=soil,
        season=season,
        organic_matter=organic_matter,
        symptoms=notes,
        language=language
    )

    # -------------------------
    # AGENT 6 → Weather Advisory
    # -------------------------
    weather_advice = agent_weather_advisory(
        crop=crop,
        location=location,
        season=season,
        temp_c=temp_c,
        humidity=humidity,
        rain_mm=rain_mm,
        wind_kmph=wind_kmph,
        language=language
    )

    # -------------------------
    # AGENT 7 → Soil Health Advisory
    # -------------------------
    soil_advice = agent_soil_advisory(
        soil_type=soil,
        moisture=moisture,
        ph=ph,
        drainage=drainage,
        organic_matter=organic_matter,
        language=language
    )

    # -------------------------
    # AGENT 8 → Organic Nutrient Booster
    # -------------------------
    organic_booster = agent_organic_nutrient_booster(
        soil_type=soil,
        organic_matter=organic_matter,
        crop=crop,
        symptoms=notes,
        language=language
    )

    # -------------------------
    # AGENT 9 → Historical Pattern Warning
    # -------------------------
    historical_warnings = agent_historical_pattern_analyzer(
        crop=crop,
        location=location,
        season=season,
        notes=notes,
        language=language
    )

    # -------------------------
    # AGENT 10 → Yield Estimator
    # -------------------------
    yield_estimate = agent_yield_estimator(
        crop=crop,
        soil_type=soil,
        season=season,
        weather_summary=f"Temp {temp_c}°C, Humidity {humidity}%, Rain {rain_mm} mm",
        symptoms=notes,
        language=language
    )

    # -------------------------
    # JSON STRUCTURED OUTPUT
    # -------------------------
    json_output = mittisense_json_report(
        crop=crop,
        location=location,
        soil=soil,
        season=season,
        notes=notes,
        language=language
    )

    # -------------------------
    # FINAL RETURN BUNDLE
    # -------------------------
    return {
        "core_advisory": core,
        "disease_expert": disease,
        "final_advisory": final_advisory,
        "short_summary": short_summary,
        "fertilizer_advice": fertilizer_advice,
        "weather_advice": weather_advice,
        "soil_advice": soil_advice,
        "organic_booster": organic_booster,
        "historical_patterns": historical_warnings,
        "yield_estimate": yield_estimate,
        "json_output": json_output
    }

# ✔️ Testing the Master Pipeline

In [ ]:
full = mittisense_master_pipeline(
    crop="Cotton",
    location="Surat, Gujarat",
    soil="Black Soil",
    season="Kharif",
    notes="Leaves have small white spots",
    temp_c=36,
    humidity=48,
    rain_mm=3,
    wind_kmph=10,
    organic_matter="Low",
    language="english"
)

print(full["final_advisory"])
print("\n--- SUMMARY ---")
print(full["short_summary"])